In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/MyDrive/GeekBrains/'


In [20]:
import pandas as pd; pd.set_option('display.max_columns', None)

import pandas as pd
import numpy as np

import dill

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn import preprocessing

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

import itertools


from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [4]:
data = pd.read_csv(path +"data_banknote_authentication.txt", header=None)
data.head(3)


,0,1,2,3,4
0,3.6216,8.6661,-2.8073,-0.44699,0
1,4.5459,8.1674,-2.4586,-1.46210,0
2,3.8660,-2.6383,1.9242,0.10645,0


In [5]:
data = data.rename(columns={0:'0', 1:'1', 2:'2', 3:'3', 4:'4'})

In [6]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [7]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]



In [8]:
continuous_columns = ['0', '1', '2', '3']

In [9]:
final_transformers = list()

    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [10]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [11]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(n_estimators=1000, learning_rate=1.0, max_depth=1, random_state=42)),
])

In [12]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('0',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='0')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('1',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='1')),
                                                                 ('scaler',
                                                                  StandardScaler())])),
                                                ('2',
                                                 Pipeline(steps=[('selector',
                                                               

In [21]:
with open(path+"boost_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)